# Logistic Regression 
그럼 가장 간단한 Linear Regression 에서는 선형적인 관계가 있는 수요와 공급, 수요와 가격 등과 같은 Linear 한 관계를 예측하기 위한 용도로 주로 사용이 된다. 하지만 Email 내용이 Spam 인지 아닌지? Boolean 형태로 예측을 하는 것과 같이 논리적인 예측을 수행하고자 한다면 Logistic 분석을 사용하여 예측 결과를 0~1 사이에 수렴하도록 처리할 필요가 있다. 
기본적으로  (1)모델을 정의, (2)Cost Function 정의, (3) Gradient Descent Algorithm 을 통한 최적의 W 값 훈련이라는 Process 는 Linear Regression 과 크게 다르지 않지만 단순한 Y = WX + b 형태의 Linear 모델을 Logistic 하게 0 ~ 1에 수렴하게 만들기 위해서 1/(1+E^WX) 와 같은 모델을 사용하기 때문에 Cost Function 또한 Cross-Entropy 라는 Function 을 사용하게 된다. 

<H3>(1) 데이터 준비</H3>   
훈련에 사용할 데이터는 아래와 같이 구성되어 있다.     
  [X 데이터]    [Y 데이터]  
[ 1.  2.  1.] [ 0.  0.  1.]  
위의 예를 보면 1,2,1 은 Y 데이터를 순서대로 A,B,C 라고 했을 때 [0, 0, 1] 임으로 C 라고 보면 된다. 

In [1]:
import tensorflow as tf
import numpy as np

# 05train.txt
# #x0 x1 x2    y[A   B   C]
# 1   2   1      0   0   1     # C
# 1   3   2      0   0   1
# 1   3   4      0   0   1
# 1   5   5      0   1   0     # B
# 1   7   5      0   1   0
# 1   2   5      0   1   0
# 1   6   6      1   0   0     # A
# 1   7   7      1   0   0

logs_path = '/tmp/tensorflow_logs/chap2-logistic'
xy = np.loadtxt('05train.txt', unpack=True, dtype='float32')

x_data = np.transpose(xy[:3])
y_data = np.transpose(xy[3:])


print('x_data : {0}'.format(x_data.shape)) 
print('x_data : {0}'.format(x_data))
print('===============================')
print('y_data : {0}'.format(y_data.shape)) 
print('y_data : {0}'.format(y_data))

x_data : (8, 3)
x_data : [[ 1.  2.  1.]
 [ 1.  3.  2.]
 [ 1.  3.  4.]
 [ 1.  5.  5.]
 [ 1.  7.  5.]
 [ 1.  2.  5.]
 [ 1.  6.  6.]
 [ 1.  7.  7.]]
y_data : (8, 3)
y_data : [[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]


<H3>(2) Graph 정의</H3>   
Linear Regression 에서 했던 것 처럼 데이터 Train 을 위한 X,Y PlaceHolder 를 정의하고 훈련할 Weight 값의 Matrix 를 0으로 초기화 하여 생성한다. 이전 예제에서는 하나의 Weight 값을 훈련하였지만 여기서 Weight Matirx 의 사이즈는 [3,3] 으로 Input X 의 컬럼 사이즈 수 곱하기 Output Y 컬럼의 사이즈 수를 곱한 사이즈가 된다.   
또 Linear Regression 과는 달리 hypothesis 는 1/(1+e^(1-WX) 를 사용하고 Cost Function은 Cross Entropy 를 사용한다는 점이 다르다. hypothesis 는 기존의 Linear 와 같이 무한대~무한대의 Y 값을 갖는 것이아닌 0~1 로 수렴하도록 하는 함수를 사용한다. Cost Function 은 당연히 hypothesis 가 다르기 때문에 달라지는데 "두 개의 확률 분포간의 차이(거리)를 나타내는 지표 차이가 클수록 큰 값이 나오고, 두 개가 같아질 때 최소값이 나온다"는 특성을 가지고 있기 때문에 Cost 함수로써 사용하기에 적합하다. 

In [2]:
X = tf.placeholder("float", [None, 3])  
Y = tf.placeholder("float", [None, 3])  

W = tf.Variable(tf.zeros([3, 3]))       

#hypothesis = tf.nn.softmax(tf.matmul(X, W))
# 로지스틱 함수 정의 
h = tf.matmul(X, W)
hypothesis = tf.div(1. , 1. + tf.exp(-h))

# cross-entropy cost 함수
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), reduction_indices=1))

learning_rate = 0.01
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

<H3>(3) Session 실행</H3>   
이제 위에 정의한 Graph 에 Data를 넣어서 훈련을 하는 일만 남았다. 데이터를 Train 하면서 200개 단위로 중간중간 Cost 값의 변화와 Weight 값의 변화를 출력한다. 전체적으로 Cost 는 감소하는 방향일 것이고, Weight 는 아까 위에서 정의한 것처럼 [3,3] Matrix 형태로 Train 이 되면서 최적의 Weight 값을 향해서 계속해서 변하게 된다. 

In [4]:
with tf.Session() as sess:
    sess.run(init)
    tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    # Train 
    for step in range(2001):
        sess.run(train, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            feed = {X: x_data, Y: y_data}
            print('{:4} {:8.6}'.format(step, sess.run(cost, feed_dict=feed)), *sess.run(W))

    # Predict 
    d = sess.run(hypothesis, feed_dict={X: [[1, 11, 7], [1, 3, 4], [1, 1, 0]]})
    print("d : ", *d, end=' ')
    print(sess.run(tf.argmax(d, 1)))                   # d :  ...  [0 1 2]


   0 0.659008 [[ 0.00125   0.001875  0.001875]
 [ 0.008125  0.00875   0.005   ]
 [ 0.008125  0.009375  0.004375]]
 200 0.0671894 [[ 0.04379454  0.07867108  0.17124081]
 [ 0.28194615  0.31851289  0.43854883]
 [ 0.28194615  0.39335516  0.35015211]]
 400 0.036633 [[ 0.052417    0.09700219  0.23702058]
 [ 0.33672115  0.37890568  0.59592026]
 [ 0.33672115  0.48501074  0.45815203]]
 600 0.0254659 [[ 0.05746159  0.10803338  0.2795071 ]
 [ 0.36866435  0.41344035  0.69473052]
 [ 0.36866435  0.5401665   0.52217656]]
 800 0.0196235 [[ 0.06104128  0.11598119  0.31128192]
 [ 0.39128733  0.43758801  0.76737934]
 [ 0.39128733  0.57990563  0.56769681]]
1000 0.0160119 [[ 0.06381883  0.12221068  0.33680922]
 [ 0.40881604  0.45612246  0.82505465]
 [ 0.40881604  0.61105251  0.6030156 ]]
1200 0.0135504 [[ 0.06608921  0.12734064  0.35821477]
 [ 0.42312837  0.47114357  0.87298703]
 [ 0.42312837  0.63670212  0.6318714 ]]
1400 0.0117611 [[ 0.06800976  0.13170502  0.37668437]
 [ 0.43522426  0.48376089  0.914053